In [1]:
import requests
import pandas as pd
import time

In [2]:
#KEYS
API_KEY = "xxxx"

#FollowGrubby
CHANNEL_ID = "UCCF6pCTGMKdo9r_kFQS-H3Q"

In [3]:
def get_video_details(video_id):
    
        #API CALL
        url_2_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&key="+API_KEY+"&part=statistics"
        response_2_stats = requests.get(url_2_stats).json()
        view_count = response_2_stats['items'][0]['statistics']['viewCount']
        like_count = response_2_stats['items'][0]['statistics']['likeCount']
        dislike_count = response_2_stats['items'][0]['statistics']['dislikeCount']
        comment_count = response_2_stats['items'][0]['statistics']['commentCount']
        
        
        return view_count,like_count,dislike_count,comment_count

In [4]:
def get_videos(df):
    pageToken = ""
    while 1:
        #APICALL
        url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000&"+pageToken

        response = requests.get(url).json()
        time.sleep(1)
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace("&amp;","")
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]
                view_count, like_count, dislike_count, comment_count = get_video_details(video_id)

                #saving the data to the dataframe
                
                df = df.append({'video_id':video_id,'video_title':video_title,
                                "upload_date":upload_date,"view_count":view_count,
                                "like_count":like_count,"dislike_count":dislike_count,
                                "comment_count":comment_count},ignore_index=True)
        try:
            if response['nextPageToken'] != None:
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break


    return df
        

In [5]:
df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','dislike_count','comment_count'])

df = get_videos(df)

In [6]:
df

,video_id,video_title,upload_date,view_count,like_count,dislike_count,comment_count
0,Z3jG0dpeAw0,2v2 WITH THEVIPER! | AoE4,2021-12-10,1816,94,2,18
1,c-cKtv_zczU,The Unfinished CLASH ft. TheViper | AoE4 | Grubby,2021-12-09,5739,193,1,17
2,KdO_zN96bis,My MAP TIERLIST! | AoE4,2021-12-09,4068,128,1,40
3,Mz5y7Adwzi4,COLLAB ft. TheViper! - AoE4 Balance Discussion...,2021-12-08,12214,460,6,83
4,msxpkiL_DsU,Testing FIRELANCERS! | AoE4 | Grubby,2021-12-07,5657,140,3,29
...,...,...,...,...,...,...,...
515,de-aCP1CXgc,Grubby | Warcraft 3 REFORGED - QA With Classic...,2018-11-03,76908,1643,30,430
516,IYHBqNnHXQ4,Grubby | Warcraft 3 TFT | 1.30 | ORC v NE on T...,2018-10-30,53466,765,19,55
517,3HGF9UghHAc,Grubby | Warcraft 3 TFT | 1.30 | HU v UD on Tw...,2018-10-30,38951,527,15,55
518,F-Wzhr7HXQQ,Grubby | Warcraft 3 TFT | 1.30 | ORC v UD on C...,2018-10-28,29752,686,15,70


In [7]:
df.to_csv('youtube_api_dataframe_2nd_pull.csv')